# Embed an executable in the final PDF

In [44]:
0x290

656

For now it's just a 'hello world' but hopefully I can get an executable with the C64 edition of Psychedelia embedded in the PDF. That way you can both read the PDF and also use it run and play the game like so:

```bash
./psychedelia_syndrome_exe.pdf
```

Create a placeholder file called `src/include_exe.tex` for the xetex build. This will reserve the bytes at the top of the PDF where we will inject our executable directly into the PDF file it builds.

In [2]:
import os
EXE_OFFSET = 0x290
LINE_WIDTH = 80
stream_len = os.stat("iridisalpha.exe").st_size - EXE_OFFSET
length_len = len(str(stream_len))

TEX = """\pdfobjcompresslevel=0
\immediate
\pdfobj
{{
<<
/Length {}
>>
stream
{}
{}
endstream
}}%
"""

# Hardcoded unfortunately, so inspect the pdf file in case it changes.
PDF_PREFIX = """%PDF-1.5
%ÐÔÅØ
1 0 obj
 << /Length {} >> stream"""

texfile = open('../src/include_exe.tex','w')

prefix_size = (len(PDF_PREFIX) - 2) + length_len
stream_len_for_prefix = stream_len - prefix_size

texfile.write(TEX.format(stream_len_for_prefix,
                        '\n'.join(["0" * (LINE_WIDTH-1)] * int(stream_len/LINE_WIDTH)),
                         "0" * (stream_len % LINE_WIDTH)
                        ))
texfile.close()

Inject our binary into the final pdf. The first 0x40 bytes are the important bit of the `ape` binary header, then we use some reserved space in the `ape` header to inject our PDF stream object containing the rest of the binary file, and then we write out the rest of the PDF file.

In [7]:
exefile = open('iridisalpha.exe','rb')
exefile.seek(0, 0)
exe_prefix =  exefile.read(EXE_OFFSET)
exefile.seek(EXE_OFFSET + prefix_size, 0)
exe_suffix = exefile.read()

pdffile = open('../out/iatheory_release.pdf','rb')

start_offset = pdffile.read().index('stream'.encode('utf-8')) + 6
pdffile.seek(0, 0)
pdf_prefix = pdffile.read(start_offset)
end_offset = pdffile.read().index('endstream'.encode('utf-8'))
pdffile.seek(end_offset, 0)
pdf_suffix = pdffile.read()

exe_pdffile = open('../out/iatheory_release_exe.pdf','wb')
exe_pdffile.write(exe_prefix)
exe_pdffile.write(pdf_prefix)
exe_pdffile.write(exe_suffix)
exe_pdffile.write(pdf_suffix)
exe_pdffile.close()

Try it out to make sure it works!

In [8]:
!evince ../out/iatheory_release_exe.pdf


(evince:999540): EvinceDocument-CRITICAL **: 21:23:07.024: ev_annotation_markup_set_label: assertion 'label != NULL' failed

(evince:999540): EvinceDocument-CRITICAL **: 21:23:07.029: ev_annotation_markup_set_label: assertion 'label != NULL' failed

(evince:999540): EvinceDocument-CRITICAL **: 21:23:07.030: ev_annotation_markup_set_label: assertion 'label != NULL' failed

(evince:999540): EvinceDocument-CRITICAL **: 21:23:07.031: ev_annotation_markup_set_label: assertion 'label != NULL' failed

(evince:999540): EvinceDocument-CRITICAL **: 21:23:07.032: ev_annotation_markup_set_label: assertion 'label != NULL' failed

(evince:999540): EvinceDocument-CRITICAL **: 21:23:07.032: ev_annotation_markup_set_label: assertion 'label != NULL' failed


In [9]:
!wine ../out/iatheory_release_exe.pdf